In [1]:
import sys
from pathlib import Path
import json

import numpy as np
import pennylane as qml
from matplotlib import pyplot as plt
from qiskit import QuantumCircuit
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.providers.aer import StatevectorSimulator
from qiskit.circuit.library import ZZFeatureMap
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.tensorboard import SummaryWriter
from pandas import DataFrame

sys.path.extend(['/home/quic/QUIC-Projects/'])
from classifiers.quantum.qml.qasvm import PseudoTensorSoftQASVM
from classifiers.datasets.sklearn import SklearnDataset
from classifiers.quantum.ansatz import MultilayerCircuit9FeatureMap
from classifiers.convex.svm import CvxSoftQASVM

In [2]:
# simulation setting
n_samples = 200
n_feature = 10
n_qubits_list = np.arange(6, 13+1, dtype=int) # 6 7 8 9 10 11 12 13
n_layers_list = np.arange(1, 19+1, 2, dtype=int) # 1 3 5 7 9 11 13 15 17 19
C=10**3
lamda=10**3
dir_name = 'numerical_simulation_bp4'

In [3]:
def load_train_data(dataset, train_size:float=2**13, reproducible_seed:int=None):
    data = dataset.data.to_numpy()
    label = dataset.target.to_numpy().astype(float)
    # label = np.where(label%2, 1, 0)
    mask = (label==0) + (label==1)
    data = data[mask]
    label = label[mask]
    if reproducible_seed is None:
        reproducible_seed = np.random.randint(99999)
    X_train, _, y_train, _ = train_test_split(data, label, train_size=train_size, test_size=1, random_state=reproducible_seed)
    return X_train, y_train, reproducible_seed

def reduce_and_normalize_data(n_components, X_train):
    scaler = StandardScaler()
    pca = PCA(n_components=n_components)
    X_train = scaler.fit_transform(X_train)
    X_train = pca.fit_transform(X_train)
    for i, d in enumerate(X_train):
        X_train[i] = d/np.linalg.norm(d)
    return X_train

def construct_training_quantum_kernel_matrix(feature_map:QuantumCircuit, X_train:np.ndarray):
    quantum_instance = QuantumInstance(backend = StatevectorSimulator(device='CPU')) # TODO: gpu vs cpu
    quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance, enforce_psd=False)
    training_kernel = quantum_kernel.evaluate(X_train, X_train)
    return training_kernel

def epsilon(fvec, true_fvec):
    return np.sqrt(np.mean(np.abs(fvec-true_fvec)**2)).item()

def make_figure(df:DataFrame):
    fig, ax = plt.subplots()
    mappable = ax.imshow(df, cmap='binary')
    ax.set_xlabel('n_qubits')
    ax.set_ylabel('n_layers')
    ax.set_xticks(np.arange(len(n_qubits_list)))
    ax.set_yticks(np.arange(len(n_layers_list)))
    ax.set_xticklabels(n_qubits_list)
    ax.set_yticklabels(n_layers_list)
    fig.colorbar(mappable)
    return fig

In [4]:
# setting
dir_path = Path.cwd() / dir_name
dir_path.mkdir(parents=True, exist_ok=True)
dataset = fetch_openml('mnist_784')
print('loaded mnist data set')

loaded mnist data set


In [5]:
from pandas import read_csv

if dir_path.joinpath('grad_exp_summary').exists():
    grad_exp_summary = read_csv(dir_path.joinpath('grad_exp_summary'), index_col=0)
    grad_exp_summary.columns = grad_exp_summary.columns.astype(int)
else:
    grad_exp_summary = DataFrame({n_qubits:dict(zip(n_layers_list, np.zeros_like(n_layers_list))) for n_qubits in n_qubits_list}, dtype=float)
if dir_path.joinpath('grad_var_summary').exists():
    grad_var_summary = read_csv(dir_path.joinpath('grad_var_summary'), index_col=0)
    grad_var_summary.columns = grad_var_summary.columns.astype(int)
else:
    grad_var_summary = DataFrame({n_qubits:dict(zip(n_layers_list, np.zeros_like(n_layers_list))) for n_qubits in n_qubits_list}, dtype=float)

# summary_writer = SummaryWriter(log_dir=dir_path)
feature_map = ZZFeatureMap(feature_dimension=n_feature, reps=2, entanglement='linear')
for n_qubits in n_qubits_list:
    sub_dir_path = dir_path / f'n_qubits={n_qubits}'
    sub_dir_path.mkdir(parents=True, exist_ok=True)
    writer = SummaryWriter(log_dir=sub_dir_path)
    train_size = 2**n_qubits

    # ansatz setup
    device:qml.Device = qml.device('lightning.qubit', wires=n_qubits) # TODO: gpu vs cpu
    def var_form(params):
        for w in device.wires:
            qml.Hadamard(wires=w)
        qml.BasicEntanglerLayers(params, wires=device.wires, rotation=qml.RY)
        # qml.StronglyEntanglingLayers(params, wires=device.wires) # TODO: var_form architechure
    for n_layers in n_layers_list:
        sub_sub_dir_path = sub_dir_path / f'n_layers={n_layers}'
        sub_sub_dir_path.mkdir(parents=True, exist_ok=True)
        if not sub_sub_dir_path.joinpath('result.json').exists():
            # data loading
            if not sub_sub_dir_path.joinpath('reference.json').exists():
                X_train, y_train, reproducible_seed = load_train_data(dataset, train_size=train_size)
                X_train = reduce_and_normalize_data(n_feature, X_train)
                train_kernel = construct_training_quantum_kernel_matrix(feature_map=feature_map, X_train=X_train)
                # save reference key
                with open(sub_sub_dir_path/'reference.json', 'w') as fp:
                    json.dump(dict(seed = reproducible_seed), fp=fp, default=list)
            else:
                with open(sub_sub_dir_path/'reference.json', 'r') as fp:
                    reference = json.load(fp=fp)
                reproducible_seed = reference['seed']
                print(f'Reference loaded for {n_qubits} qubits {n_layers} layers (seed: {reproducible_seed})')
                X_train, y_train, _ = load_train_data(dataset, train_size=train_size, reproducible_seed=reproducible_seed)
                X_train = reduce_and_normalize_data(n_feature, X_train)
                train_kernel = construct_training_quantum_kernel_matrix(feature_map=feature_map, X_train=X_train)
            print(f'{train_size} X {train_size} train kernel matrix generated for {n_layers} layers (seed: {reproducible_seed})')
            # gradient measurement
            parameter_shape = qml.BasicEntanglerLayers.shape(n_layers=n_layers, n_wires=device.num_wires)
            # parameter_shape = qml.StronglyEntanglingLayers.shape(n_layers=n_layers, n_wires=device.num_wires) # TODO: var_form architechure
            qasvm = PseudoTensorSoftQASVM(data=train_kernel, label=y_train, C=C, lamda=lamda, device=device, feature_map=None, var_form=var_form)
            # grad_results = np.empty(shape=(n_samples, *parameter_shape))
            grad_results = np.empty(shape=(n_samples,))
            # grad_fn = qml.jacobian(qasvm.cost_fn)
            # grad_fn = qml.jacobian(qasvm.avg_f)
            for s in range(n_samples):
                params=qml.numpy.random.random(parameter_shape, requires_grad=True)
                grad_results[s] = qasvm.grad_fn_00(params)
            # grad_results_00 = grad_results[:, 0, 0]
            grad_results_00 = grad_results[:]
            grad_exp = grad_results_00.mean()
            grad_var = grad_results_00.var()
            grad_exp_summary[n_qubits][n_layers] = grad_exp
            grad_var_summary[n_qubits][n_layers] = grad_var
            writer.add_scalar('Grad/Expectation', grad_exp, n_layers)
            writer.add_scalar('Grad/Variance', grad_var, n_layers)
            writer.add_scalar('Log10Grad/AbsExpectation', np.log10(np.abs(grad_exp)), n_layers)
            writer.add_scalar('Log10Grad/Variance', np.log10(grad_var), n_layers)
            # save result
            with open(sub_sub_dir_path/'result.json', 'w') as fp:
                json.dump(dict(grad_results = grad_results.tolist(), sample_size=n_samples), fp=fp, default=list)
            grad_exp_summary.to_csv(dir_path/'grad_exp_summary')
            grad_var_summary.to_csv(dir_path/'grad_var_summary')
        else:
            print(f'Already done for {n_qubits} qubits {n_layers} layers')

64 X 64 train kernel matrix generated for 1 layers (seed: 35098)
64 X 64 train kernel matrix generated for 3 layers (seed: 40978)
64 X 64 train kernel matrix generated for 5 layers (seed: 22574)
64 X 64 train kernel matrix generated for 7 layers (seed: 75072)
64 X 64 train kernel matrix generated for 9 layers (seed: 88867)
64 X 64 train kernel matrix generated for 11 layers (seed: 34037)
64 X 64 train kernel matrix generated for 13 layers (seed: 73726)
64 X 64 train kernel matrix generated for 15 layers (seed: 29477)
64 X 64 train kernel matrix generated for 17 layers (seed: 5095)
64 X 64 train kernel matrix generated for 19 layers (seed: 86038)
128 X 128 train kernel matrix generated for 1 layers (seed: 30510)
128 X 128 train kernel matrix generated for 3 layers (seed: 90519)
128 X 128 train kernel matrix generated for 5 layers (seed: 36372)
128 X 128 train kernel matrix generated for 7 layers (seed: 76106)
128 X 128 train kernel matrix generated for 9 layers (seed: 6266)
128 X 128 tr